In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz, process

def make_choropleths(df, country_col, col1, col2, shapefile_path):
    # Load shapefile
    world = gpd.read_file(shapefile_path)
    shapefile_countries = world['ADMIN'].unique()

    # Match country names
    def match_country(name):
        match = process.extractOne(name, shapefile_countries, scorer=fuzz.partial_ratio)
        if match and match[1] > 80:
            return match[0]
        return None

    df = df.copy()
    df['Mapped Country'] = df[country_col].apply(match_country)

    # Merge
    merged = world.merge(df, how="left", left_on="ADMIN", right_on="Mapped Country")

    # Plotting
    sns.set_theme(style="whitegrid")
    plt.rcParams['font.family'] = 'Georgia'
    plt.rcParams['axes.facecolor'] = '#f5f5f5'

    fig, ax = plt.subplots(1, 2, figsize=(20, 12))

    for i, column in enumerate([col1, col2]):
        merged.boundary.plot(ax=ax[i], linewidth=0.8, color='black')
        merged.plot(
            column=column,
            ax=ax[i],
            legend=True,
            legend_kwds={'label': column, 'orientation': "horizontal"},
            cmap='YlGnBu' if i else 'YlOrRd',
            missing_kwds={"color": "lightgrey", "label": "No data"}
        )
        ax[i].set_title(f"{column} by Country", fontsize=18, fontweight='bold')
        ax[i].axis('off')

    plt.tight_layout()
    plt.show()

c:\Users\ryant\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
shapefile_path = "data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"
